In [ ]:
def freq_reduction(df, new_freq):
    mean_df = pd.DataFrame()

    for i in range(0, df.shape[1], new_freq):
        # Taking the mean of every 500 columns
        mean_cols = df.iloc[:, i:i+new_freq].mean(axis=1)
        # Appending the new columns to mean_df
        mean_df = pd.concat([mean_df, mean_cols], axis=1)
        
    return mean_df

In [ ]:
collected_x_train = []
collected_x_test = []

lower_freq_x = freq_reduction(pd.DataFrame(My_X_test_n_rs), 50).to_numpy()

for i in range(6):
    collected_x_train.append(lower_freq_x[4*i].tolist().append(i+1))
    collected_x_train.append(lower_freq_x[4*i+3].tolist().append(i+1))
    collected_x_test.append(lower_freq_x[4*i+2].tolist())
    collected_x_test.append(lower_freq_x[4*i+1].tolist())

first_x_train = pd.DataFrame(collected_x_train[0:2]).to_csv()
second_x_train = pd.DataFrame(collected_x_train[2:4]).to_csv()
third_x_train = pd.DataFrame(collected_x_train[4:6]).to_csv()
fourth_x_train = pd.DataFrame(collected_x_train[6:8]).to_csv()
fifth_x_train = pd.DataFrame(collected_x_train[8:10]).to_csv()
sixth_x_train = pd.DataFrame(collected_x_train[10:12]).to_csv()


In [ ]:
total_accuracies = 0
total_iterations = 0
num_test_splits = 2

for i in range(num_test_splits):
    lot_size = 12//num_test_splits
    csv_tsfel_test = pd.DataFrame(collected_x_test[lot_size*i:lot_size*(i+1)]).to_csv()
    
    query = f'''
    You are an activity classification model.
    
    **Instructions:**
    1. **Train** a Decision Tree model using the following training data provided in csv format, which includes 6 classes with 30 feature columns and 1 label column at the end.
    2. **Predict** the class for each row in the test data, which consists of {lot_size} rows with 30 feature columns only. The label has to be predicted.
    3. **Output**: Provide only the {lot_size} predicted activity labels as a single line of space-separated integers. Do not include any additional text, explanations, or code.
    
    **Training Data:** for first class
    {first_x_train}

    **Training Data:** for second class
    {second_x_train}

    **Training Data:** for third class
    {third_x_train}

    **Training Data:** for fourth class
    {fourth_x_train}

    **Training Data:** for fifth class
    {fifth_x_train}

    **Training Data:** for sixth class
    {sixth_x_train}
    
    **Test Data:**
    {csv_tsfel_test}
    '''

    # To use Groq LLMs 
    model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
    toggle = False
    while not toggle:
        try:
            llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=1)
            answer = llm.invoke(query)
            # print(f'Correct Answer was {y_test}')
            
            
            answer = list(map(int, answer.content.split()))
            res = 0
            correct_answers = [3*i+1,3*i+1,3*i+2,3*i+2,3*i+3,3*i+3]
            print(f"Correct answer was: {correct_answers}")
            print(f'Model predicted: {answer}')
            
            for j in range(lot_size):
                if correct_answers[j] == answer[j]:
                    res += 1
            accuracy = res/lot_size
            
            total_accuracies += accuracy
            total_iterations += 1
            toggle = True
        except Exception as e:
            print(f"An error occurred: {e}")
print(f'\nTotal Accuracy turned out to be {100*total_accuracies/total_iterations}%')

Correct answer was: [1, 1, 2, 2, 3, 3]
Model predicted: [1, 2, 3, 4, 5, 6]
Correct answer was: [4, 4, 5, 5, 6, 6]
Model predicted: [1, 2, 3, 4, 5, 6]

Total Accuracy turned out to be 16.666666666666664%
